In [1]:
import processing as llm
import database as db

In [2]:
import pandas as pd
# Simulate output from ocr
receipt = pd.read_csv('../data/scanned_rewe_receipts_cleaned.csv', index_col=0).query("receipt_id == 'Rewe_1.jpg'").rename(columns={'product_name':'product_abbr'})


In [3]:
receipt

,product_abbr,price,receipt_id
0,VOLLKORNTORTILLA,2.58,Rewe_1.jpg
1,TOFU NATUR,4.38,Rewe_1.jpg
2,HAUCHSCHN CURRY,3.18,Rewe_1.jpg
3,HONEYPOMELO PINK,2.99,Rewe_1.jpg
4,DINKEL - CRUNCHY,2.59,Rewe_1.jpg
5,TARTEX SC TOSKA,2.69,Rewe_1.jpg
6,AUFSTRICH PAP . ZU,2.38,Rewe_1.jpg
7,BROT VITAL GF,5.98,Rewe_1.jpg
8,INST . NUDELGER . HU,2.07,Rewe_1.jpg
9,INST . NUDELG . ENTE,1.38,Rewe_1.jpg


# Process a receipt with Mistral

In [4]:
llm_processed_receipt = llm.process_receipt(receipt)

Requesting Mixtral for VOLLKORNTORTILLA…
Received response
Parses response successfully, Vollkorn Tortilla
Requesting Mixtral for TOFU NATUR…
Received response
Parses response successfully, Tofu Natur
Requesting Mixtral for HAUCHSCHN CURRY…
Received response
Parses response successfully, Rügenwalder Mühle Veganer Hauchschnitt Typ Hähnchen
Requesting Mixtral for HONEYPOMELO PINK…
Received response
Parses response successfully, Pink Honey Pomelo
Requesting Mixtral for DINKEL - CRUNCHY…
Received response
Parses response successfully, Dinkel Crunchy
Requesting Mixtral for TARTEX SC TOSKA…
Received response
Parses response successfully, Tartex Sc Toskana
Requesting Mixtral for AUFSTRICH PAP . ZU…
Received response
Parses response successfully, Aufstrich Paprika Zucchini
Requesting Mixtral for BROT VITAL GF…
Received response
Parses response successfully, Vollkornbrot Vital (glutenfrei)
Requesting Mixtral for INST . NUDELGER . HU…
Received response
Parses response successfully, Instant Nudel

In [5]:
llm_processed_receipt

,product_abbr,price,receipt_id,productName,categoryMain,categorySub,embedding
0,VOLLKORNTORTILLA,2.58,Rewe_1.jpg,Vollkorn Tortilla,"Brot, Cerealien & Aufstriche",Backwaren,"[-0.045166015625, 0.00507354736328125, 0.03604..."
1,TOFU NATUR,4.38,Rewe_1.jpg,Tofu Natur,Fleisch & Fisch,Fleischalternativen,"[-0.03302001953125, 0.0308990478515625, 0.0634..."
2,HAUCHSCHN CURRY,3.18,Rewe_1.jpg,Rügenwalder Mühle Veganer Hauchschnitt Typ Häh...,Fleisch & Fisch,Fleischalternativen,"[-0.025238037109375, 0.0048675537109375, 0.080..."
3,HONEYPOMELO PINK,2.99,Rewe_1.jpg,Pink Honey Pomelo,Obst & Gemüse,Frisches Obst,"[-0.0192413330078125, 6.914138793945312e-05, 0..."
4,DINKEL - CRUNCHY,2.59,Rewe_1.jpg,Dinkel Crunchy,"Brot, Cerealien & Aufstriche",Müsli & Cerealien,"[0.0006494522094726562, 0.021484375, 0.0436706..."
5,TARTEX SC TOSKA,2.69,Rewe_1.jpg,Tartex Sc Toskana,Fertiggerichte & Konserven,Fixprodukte,"[-0.0229949951171875, 0.0239715576171875, 0.05..."
6,AUFSTRICH PAP . ZU,2.38,Rewe_1.jpg,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.050994873046875, 0.0175323486328125, 0.041..."
7,BROT VITAL GF,5.98,Rewe_1.jpg,Vollkornbrot Vital (glutenfrei),"Brot, Cerealien & Aufstriche",Brot,"[-0.02496337890625, 0.0084991455078125, 0.0291..."
8,INST . NUDELGER . HU,2.07,Rewe_1.jpg,Instant Nudelgericht Huhn,Fertiggerichte & Konserven,Fertiggerichte,"[-0.04742431640625, -0.01116180419921875, 0.06..."
9,INST . NUDELG . ENTE,1.38,Rewe_1.jpg,Instant Nudelgericht Ente,Fertiggerichte & Konserven,Fertiggerichte,"[-0.0391845703125, -0.00612640380859375, 0.055..."


# Write to database

In [6]:
# Run setup (only needs to run once)
db.setup()

Created table receipts
Created table rewe


In [7]:
db.insert_receipt_data(llm_processed_receipt)

Wrote in database.


In [8]:
# Retrieve all receipts data
df = db.data()

In [10]:
df

,id_pk,receipt_id,price,product_abbr,product_name,category_main,category_sub,embedding
0,1,Rewe_1.jpg,2.58,VOLLKORNTORTILLA,Vollkorn Tortilla,"Brot, Cerealien & Aufstriche",Backwaren,"[-0.045166016, 0.0050735474, 0.03604126, -0.00..."
1,2,Rewe_1.jpg,4.38,TOFU NATUR,Tofu Natur,Fleisch & Fisch,Fleischalternativen,"[-0.03302002, 0.030899048, 0.06341553, -0.0038..."
2,3,Rewe_1.jpg,3.18,HAUCHSCHN CURRY,Rügenwalder Mühle Veganer Hauchschnitt Typ Häh...,Fleisch & Fisch,Fleischalternativen,"[-0.025238037, 0.0048675537, 0.080444336, -0.0..."
3,4,Rewe_1.jpg,2.99,HONEYPOMELO PINK,Pink Honey Pomelo,Obst & Gemüse,Frisches Obst,"[-0.019241333, 6.914139e-05, 0.05215454, 0.001..."
4,5,Rewe_1.jpg,2.59,DINKEL - CRUNCHY,Dinkel Crunchy,"Brot, Cerealien & Aufstriche",Müsli & Cerealien,"[0.0006494522, 0.021484375, 0.043670654, 0.007..."
5,6,Rewe_1.jpg,2.69,TARTEX SC TOSKA,Tartex Sc Toskana,Fertiggerichte & Konserven,Fixprodukte,"[-0.022994995, 0.023971558, 0.05508423, -0.021..."
6,7,Rewe_1.jpg,2.38,AUFSTRICH PAP . ZU,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.050994873, 0.017532349, 0.041900635, 0.005..."
7,8,Rewe_1.jpg,5.98,BROT VITAL GF,Vollkornbrot Vital (glutenfrei),"Brot, Cerealien & Aufstriche",Brot,"[-0.024963379, 0.0084991455, 0.029159546, -0.0..."
8,9,Rewe_1.jpg,2.07,INST . NUDELGER . HU,Instant Nudelgericht Huhn,Fertiggerichte & Konserven,Fertiggerichte,"[-0.047424316, -0.011161804, 0.06591797, 0.004..."
9,10,Rewe_1.jpg,1.38,INST . NUDELG . ENTE,Instant Nudelgericht Ente,Fertiggerichte & Konserven,Fertiggerichte,"[-0.03918457, -0.006126404, 0.05596924, -0.004..."


In [14]:
# User search query
query_user_input, n_results, query_table = (['vegan'], 10, 'receipts')

query_embedding = llm.get_embeddings_by_chunks(query_user_input, 1)

query_results = db.search(query_embedding, n_results, query_table)

In [15]:
query_results

,id_pk,receipt_id,price,product_abbr,product_name,category_main,category_sub,embedding
0,3,Rewe_1.jpg,3.18,HAUCHSCHN CURRY,Rügenwalder Mühle Veganer Hauchschnitt Typ Häh...,Fleisch & Fisch,Fleischalternativen,"[-0.025238037, 0.0048675537, 0.080444336, -0.0..."
1,2,Rewe_1.jpg,4.38,TOFU NATUR,Tofu Natur,Fleisch & Fisch,Fleischalternativen,"[-0.03302002, 0.030899048, 0.06341553, -0.0038..."
2,8,Rewe_1.jpg,5.98,BROT VITAL GF,Vollkornbrot Vital (glutenfrei),"Brot, Cerealien & Aufstriche",Brot,"[-0.024963379, 0.0084991455, 0.029159546, -0.0..."
3,7,Rewe_1.jpg,2.38,AUFSTRICH PAP . ZU,Aufstrich Paprika Zucchini,"Brot, Cerealien & Aufstriche",Herzhafte Brotaufstriche,"[-0.050994873, 0.017532349, 0.041900635, 0.005..."
4,10,Rewe_1.jpg,1.38,INST . NUDELG . ENTE,Instant Nudelgericht Ente,Fertiggerichte & Konserven,Fertiggerichte,"[-0.03918457, -0.006126404, 0.05596924, -0.004..."
5,4,Rewe_1.jpg,2.99,HONEYPOMELO PINK,Pink Honey Pomelo,Obst & Gemüse,Frisches Obst,"[-0.019241333, 6.914139e-05, 0.05215454, 0.001..."
6,1,Rewe_1.jpg,2.58,VOLLKORNTORTILLA,Vollkorn Tortilla,"Brot, Cerealien & Aufstriche",Backwaren,"[-0.045166016, 0.0050735474, 0.03604126, -0.00..."
7,9,Rewe_1.jpg,2.07,INST . NUDELGER . HU,Instant Nudelgericht Huhn,Fertiggerichte & Konserven,Fertiggerichte,"[-0.047424316, -0.011161804, 0.06591797, 0.004..."
8,5,Rewe_1.jpg,2.59,DINKEL - CRUNCHY,Dinkel Crunchy,"Brot, Cerealien & Aufstriche",Müsli & Cerealien,"[0.0006494522, 0.021484375, 0.043670654, 0.007..."
9,11,Rewe_1.jpg,-0.25,LEERGUT EINWEG,Leergut Einweg,Sonstige Positionen,Pfand & Leergut,"[-0.019210815, 0.040618896, 0.015350342, 0.024..."
